## This section aims to assess and compare the performances of the models implemented in the sections 4.1, 4.2 and 4.3

In [1]:
library(randomForest)
library(caret)
library(e1071)
validation_set <- read.csv("../Data/PreProcess/processed_verification_data_split.csv") # load validation set
column_to_drop<-c("X.1","X")
# Dropped "X.1","X" because they just represent the row numbers
validation_set<-validation_set[,!(names(validation_set) %in% column_to_drop)] # drop the desired columns on validation set

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: lattice

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin




In [2]:
load("4-Models/random_forest.RData") # load Random Forest trained model -> name : classifierRF
pred <- predict(classifierRF,validation_set) # Random Forest model prediction on validation set

#table(observed=validation_set$id,predicted=pred) # Random Forest Confusion Matrix on validation set
cmRF <- confusionMatrix(data=pred, reference = factor(validation_set$id)) # Random Forest Confusion Matrix on validation set


kpRF <- cmRF$overall["Kappa"] # Cohen's Kappa coefficient, allows to indicate the level of agreement between prediction and expected
                          # Allows to take into account the proportion of false negative https://thenewstack.io/cohens-kappa-what-it-is-when-to-use-it-and-how-to-avoid-its-pitfalls/
acRF <- cmRF$overall["Accuracy"] # Accuracy (CM diagonal)

cmRF$table # Confusion Matrix displayed 

ERROR: Error in eval(predvars, data, env): objet 'colored' introuvable


## Support Vector Machine performance and cross-validation

## Accuracy

In [6]:
load("4-Models/svm_poly_classifier.RData") # loads SVM classifier -> name: svm_model_poly

pred <- predict(svm_model_poly, validation_set, probability=TRUE) # SVM prediction on validation set

In [7]:
cmSVM <- confusionMatrix(data=pred, reference=factor(validation_set$id[-1]))

kpSVM <- cmSVM$overall["Kappa"] 

acSVM <- cmSVM$overall["Accuracy"]

cmSVM$table
acSVM
kpSVM

                         Reference
Prediction                functional functional needs repair non functional
  functional                    5560                     709           2191
  functional needs repair         29                      29             24
  non functional                 934                     179           2455

Accuracy 
0.6642444

Kappa 
0.3411024

Around 66% of accuracy for the SVM classification. There are issues with the "functional needs repair" class.

## Cross-validation on training set

In [8]:
training_set <- read.csv("../Data/PreProcess/processed_training_data_split.csv")
column_to_drop<-c("X.1","X")
training_set<-training_set[,!(names(training_set) %in% column_to_drop)] # drop the desired columns
training_set$id <- factor(training_set$id)

In [9]:
# Best parameters obtained so far
tuned <- tune.svm(id~., data = training_set, kernel="polynomial", cost=10, scale=FALSE, probability=TRUE, tunecontrol=tune.control(cross=10))
summary(tuned)